# Train segmentor

This step is optional, the segmentor has already been trained and has checkpoints available. If you don't wish to train the segmentor and detector again, you can go to *[Combined training example](#section-combined)*%autocall section.

**create a training configuration**

In [28]:
import yaml
from pprint import pprint

config_file_path = './xami_model/train/segmentor_config.yaml'
with open(config_file_path, 'r') as file:
     config = yaml.safe_load(file)

print('Initial config:')
pprint(config)
config['device_id'] = 1
config['input_dir'] = './data/xami_dataset/'
config['mobile_sam_checkpoint'] = './xami_model/train/weights/sam_weights/original_mobile_sam.pt'

# save the configuration
with open(config_file_path, 'w') as file:
     yaml.safe_dump(config, file)
print(f"Updated configuration saved to {config_file_path}.")

Initial config:
{'cuda_visible_devices': '0, 1, 2, 3',
 'device_id': 1,
 'final_lr': '6e-5',
 'initial_batch_size': 4,
 'input_dir': './data/xami_dataset/',
 'kfold_iter': 0,
 'learning_rate': '3e-4',
 'mobile_sam_checkpoint': './xami_model/train/weights/sam_weights/original_mobile_sam.pt',
 'n_epochs_stop': 15,
 'num_epochs': 60,
 'total_steps': 16,
 'use_CR': True,
 'use_lr_initial_decay': True,
 'wandb_track': True,
 'weight_decay': 0.0005,
 'work_dir': './output_sam'}
Updated configuration saved to ./xami_model/train/segmentor_config.yaml.


**run training**

In [33]:
# replace the python executable with the environment where xami packages are installed
!/root/miniforge3/envs/xami_model_env_2/bin/python ./xami_model/train/train_sam.py ./xami_model/train/segmentor_config.yaml

Working directory: ./output_sam_11
Device: cuda:1
wandb: Currently logged in as: iuliaelisa15. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /workspace/raid/OM_DeepLearning/XAMI-model/wandb/run-20240611_003942-ntrqkbae
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sam_0_2024-06-11 00:39:42.773108
wandb: ⭐️ View project at https://wandb.ai/iuliaelisa15/sam
wandb: 🚀 View run at https://wandb.ai/iuliaelisa15/sam/runs/ntrqkbae/workspace
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
🚀  Training Sam with 790 training images and 264 validation images.
🚀  Training for 60 epochs with effective batch size 8 and learning rate 0.0003.
🚀  Initial learning rate: 0.0003. Final learning rate: 6e-05 after 16 steps. Weight decay: 0.0005.
🚀  Using learning rate initial decay sche

KeyboardInterrupt: 